In [7]:
pip show Flask Flask-Security


Name: Flask
Version: 3.0.0
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: C:\Users\ayush\anaconda3\Lib\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: flask-babel, Flask-BabelEx, Flask-Login, Flask-Mail, Flask-Principal, Flask-Security, Flask-SQLAlchemy, Flask-WTF
---
Name: Flask-Security
Version: 3.0.0
Summary: Simple security for Flask apps.
Home-page: https://github.com/mattupstate/flask-security
Author: Matt Wright
Author-email: matt@nobien.net
License: MIT
Location: C:\Users\ayush\anaconda3\Lib\site-packages
Requires: Flask, Flask-BabelEx, Flask-Login, Flask-Mail, Flask-Principal, Flask-WTF, itsdangerous, passlib
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, jsonify, send_file, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from flask_security import Security, SQLAlchemyUserDatastore, UserMixin, RoleMixin
from flask_mail import Mail, Message
import os
from werkzeug.utils import secure_filename
import hashlib
from datetime import datetime, timedelta

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///file_sharing.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = '7289081815'
app.config['SECURITY_PASSWORD_SALT'] = 'salt'
app.config['SECURITY_REGISTERABLE'] = True
app.config['SECURITY_SEND_REGISTER_EMAIL'] = True
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'kabirsharma7480@gmail.com'
app.config['MAIL_PASSWORD'] = 'jdjndjd'
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['ALLOWED_EXTENSIONS'] = {'pptx', 'docx', 'xlsx'}
app.config['DOWNLOAD_LINK_EXPIRY'] = 3600  # 1 hour

db = SQLAlchemy(app)
mail = Mail(app)

roles_users = db.Table(
    'roles_users',
    db.Column('user_id', db.Integer(), db.ForeignKey('user.id')),
    db.Column('role_id', db.Integer(), db.ForeignKey('role.id'))
)

class Role(db.Model, RoleMixin):
    id = db.Column(db.Integer(), primary_key=True)
    name = db.Column(db.String(80), unique=True)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    password = db.Column(db.String(255))
    active = db.Column(db.Boolean())
    roles = db.relationship('Role', secondary=roles_users, backref=db.backref('users', lazy='dynamic'))

class File(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    filename = db.Column(db.String(255))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    user = db.relationship('User', backref=db.backref('files', lazy='dynamic'))

class DownloadLink(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    token = db.Column(db.String(255), unique=True)
    file_id = db.Column(db.Integer, db.ForeignKey('file.id'))
    file = db.relationship('File', backref=db.backref('download_links', lazy='dynamic'))
    expiration_time = db.Column(db.DateTime)

user_datastore = SQLAlchemyUserDatastore(db, User, Role)
security = Security(app, user_datastore)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

def generate_download_link(file_id):
    file = File.query.get(file_id)
    if file:
        token_data = f"{file.id}-{file.filename}-{datetime.utcnow()}"
        token = hashlib.sha256(token_data.encode()).hexdigest()
        expiration_time = datetime.utcnow() + timedelta(seconds=app.config['DOWNLOAD_LINK_EXPIRY'])
        download_link = DownloadLink(token=token, file=file, expiration_time=expiration_time)
        db.session.add(download_link)
        db.session.commit()
        return token
    else:
        return None

@app.route('/ops/login', methods=['POST'])
def ops_login():
    return "Ops User Login"

@app.route('/ops/upload', methods=['POST'])
def ops_upload():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        new_file = File(filename=filename, user_id=1)
        db.session.add(new_file)
        db.session.commit()

        return jsonify({'message': 'File uploaded successfully'})
    else:
        return jsonify({'error': 'Invalid file format'})

@app.route('/client/signup', methods=['POST'])
def client_signup():
    email = request.form.get('email')
    password = request.form.get('password')

    user = user_datastore.create_user(email=email, password=security.encrypt_password(password))
    db.session.commit()

    send_verification_email(user)

    return "Client User Signup Successful"

def send_verification_email(user):
    token = security.generate_confirmation_token(user)
    confirm_url = url_for('client_verify', token=token, _external=True)

    msg = Message('Verify Your Email', recipients=[user.email])
    msg.body = f'Thank you for signing up! Please click the following link to verify your email: {confirm_url}'
    mail.send(msg)

@app.route('/client/verify/<token>', methods=['GET'])
def client_verify(token):
    user = security.confirm_token(token)

    if user:
        user.active = True
        db.session.commit()
        return "Email verification successful"
    else:
        return "Invalid verification token"

@app.route('/client/login', methods=['POST'])
def client_login():
    email = request.form.get('email')
    password = request.form.get('password')

    user = user_datastore.get_user(email)
    if user and security.check_password_hash(user.password, password):
        return "Client User Login Successful"
    else:
        return "Invalid credentials"

@app.route('/client/download/<file_id>', methods=['GET'])
def client_download(file_id):
    file = File.query.get(file_id)
    if file:
        token = generate_download_link(file.id)
        if token:
            download_url = url_for('download_file', token=token, _external=True)
            return jsonify({'download-link': download_url, 'message': 'success'})
        else:
            return jsonify({'error': 'Failed to generate download link'})
    else:
        return jsonify({'error': 'File not found'})

@app.route('/download/<token>', methods=['GET'])
def download_file(token):
    link = DownloadLink.query.filter_by(token=token).first()
    if link and link.expiration_time > datetime.utcnow():
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], link.file.filename)
        return send_file(file_path, as_attachment=True)
    else:
        return jsonify({'error': 'Invalid or expired download link'})

@app.route('/client/files', methods=['GET'])
def client_list_files():
    files = File.query.all()
    file_list = [{'id': file.id, 'filename': file.filename} for file in files]
    return jsonify({'files': file_list})

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True)


ImportError: cannot import name '_request_ctx_stack' from 'flask' (C:\Users\ayush\anaconda3\Lib\site-packages\flask\__init__.py)

In [5]:
pip install --upgrade Werkzeug

Note: you may need to restart the kernel to use updated packages.
